Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [12]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [13]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [14]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [29]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, depth], 
                            stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], 
                            stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(
        tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                            stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [30]:
num_steps = 8001

def run_graph(graph, batched_valid_and_test=False):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('Initialized')
        non_training_dict = {}
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 1000 == 0):
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                if batched_valid_and_test:
                    valid_accs = []
                    for batch_n in range(len(valid_dataset)//batch_size):
                        offset = (batch_n * batch_size)
                        batch_data = valid_dataset[offset:(offset + batch_size), :, :, :]
                        batch_labels = valid_labels[offset:(offset + batch_size), :]
                        feed_dict = {tf_valid_dataset : batch_data}
                        
                        valid_accs.append(accuracy(session.run(valid_prediction, feed_dict=feed_dict), batch_labels))
                    valid_acc = np.mean(valid_accs)
                else:
                    valid_acc = accuracy(session.run(valid_prediction, feed_dict={}), valid_labels)
                print('Validation accuracy: %.1f%%' % valid_acc)
        if batched_valid_and_test:
            test_accs = []
            for batch_n in range(len(test_dataset)//batch_size):
                offset = (batch_n * batch_size)
                batch_data = test_dataset[offset:(offset + batch_size), :, :, :]
                batch_labels = test_labels[offset:(offset + batch_size), :]
                feed_dict = {tf_test_dataset : batch_data}

                test_accs.append(accuracy(session.run(test_prediction, feed_dict=feed_dict), batch_labels))
            test_acc = np.mean(test_accs)
        else:
            test_acc = accuracy(session.run(test_prediction, feed_dict={}), test_labels)
        print('Test accuracy: %.1f%%' % test_acc)
run_graph(graph)

Initialized
Minibatch loss at step 0: 3.248662
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.830804
Minibatch accuracy: 75.0%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.321504
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 0.676590
Minibatch accuracy: 75.0%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 1.386534
Minibatch accuracy: 75.0%
Validation accuracy: 87.0%
Minibatch loss at step 5000: 0.422354
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 6000: 1.264927
Minibatch accuracy: 68.8%
Validation accuracy: 86.8%
Minibatch loss at step 7000: 0.491627
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 8000: 0.270384
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Test accuracy: 93.8%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [31]:
max_pool_graph = tf.Graph()

with max_pool_graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, depth], 
                            stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], 
                            stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # with padding SAME:
    layer3_weights = tf.Variable(
        tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                            stddev=0.1))
    # with padding VALID:
    #size_valid = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    #layer3_weights = tf.Variable(
    #    tf.truncated_normal(
    #        [size_valid * size_valid * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [32]:
run_graph(max_pool_graph)

Initialized
Minibatch loss at step 0: 3.313531
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.787105
Minibatch accuracy: 68.8%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.248685
Minibatch accuracy: 93.8%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.665659
Minibatch accuracy: 75.0%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 1.309809
Minibatch accuracy: 75.0%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.470222
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 6000: 1.047400
Minibatch accuracy: 68.8%
Validation accuracy: 87.8%
Minibatch loss at step 7000: 0.483890
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 8000: 0.172545
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Test accuracy: 94.2%


- Max pooling with stride 2, kernel size 2, and padding SAME gave below 1% accuracy boost.
- With padding VALID the accuracy was worse than before.

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [83]:
batch_size = 100
patch_size = 3
layer1_depth = 6
layer2_depth = 16
layer3_depth = 120
num_hidden = 256

final_graph = tf.Graph()

with final_graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # conv (3x3), stride 1 -> relu -> pool (kernel:2, stride:2)
    layer1_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, layer1_depth], 
                            stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([layer1_depth]))
    # conv (3x3), stride 1 -> relu -> pool (kernel:2, stride:2)
    layer2_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, layer1_depth, layer2_depth], 
                            stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[layer2_depth]))
    # conv (3x3), stride 1 -> relu -> pool (kernel:2, stride:2)
    layer3_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, layer2_depth, layer3_depth], 
                            stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[layer3_depth]))
    # conv (3x3), stride 1 -> relu -> pool (kernel:2, stride: 2)
    layer4_weights = tf.Variable(
        tf.truncated_normal([4 * 4 * layer3_depth, num_hidden], 
                            stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    # fully connected
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data, train=False):
        conv = tf.nn.conv2d(data, layer1_kernel, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_kernel, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool, layer3_kernel, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer3_biases)
        pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        fully_connected = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
        logits = tf.matmul(fully_connected, layer5_weights) + layer5_biases
        return logits
  
    # Training computation.
    logits = model(tf_train_dataset, train=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [84]:
run_graph(final_graph, batched_valid_and_test=False)

Initialized
Minibatch loss at step 0: 7.890035
Minibatch accuracy: 9.0%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.634369
Minibatch accuracy: 79.0%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.467059
Minibatch accuracy: 88.0%
Validation accuracy: 87.6%
Minibatch loss at step 3000: 0.426319
Minibatch accuracy: 86.0%
Validation accuracy: 88.7%
Minibatch loss at step 4000: 0.321018
Minibatch accuracy: 92.0%
Validation accuracy: 89.3%
Minibatch loss at step 5000: 0.397659
Minibatch accuracy: 86.0%
Validation accuracy: 89.9%
Minibatch loss at step 6000: 0.277665
Minibatch accuracy: 92.0%
Validation accuracy: 90.2%
Minibatch loss at step 7000: 0.233348
Minibatch accuracy: 92.0%
Validation accuracy: 90.6%
Minibatch loss at step 8000: 0.189876
Minibatch accuracy: 95.0%
Validation accuracy: 90.6%
Test accuracy: 96.1%


- Just adding dropout made the accuracy worse.
- Learning rate decay with `tf.train.exponential_decay(learning_rate, global_step, 100, 0.96, staircase=True)` did not help
- AdamOptimizer (default params) with dropout was not better
- removed dropout and kept Adam, accuracy rose to above 94 %.
- added L2 loss regulariser, better than dropout, but did not improve much
- tried moving the pooling after the activation function, no significant change (think this is the convention)
- replaced `max_pool` by `avg_pool`, no significant change
- add dropout before final layer, no significant change
- increase batch size to 50, about 1 % higher accuracy
- increase batch size 100, move dropout before the final layer, no significant change
- removed dropout for the 4 layer network, 1 % accuracy boost (~96%)
 - network is not complicated enough for the dropout to work properly
- tried with deeper layers with larger dens layer (256) (appr same accuracy ~96%)
 - 64 feature maps (depth) in first conv layer
 - 32 feature maps in second conv layer
- feature maps of 6 and 16 and dense layer of 500 connections gave appr. same accuracy
- added extra conv and pool layer with relu with depth 32, not much difference
- changed depth to 120, not much difference

In [67]:
batch_size = 100
patch_size = 3
layer1_depth = 6
layer2_depth = 16
layer3_depth = 120
num_hidden = 84

lenet_graph = tf.Graph()

with lenet_graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # conv (1x1) -> relu -> pool (kernel:2, stride:2)
    layer1_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, layer1_depth], 
                            stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([layer1_depth]))
    # conv (1x1) -> relu -> pool (kernel:2, stride:2)
    layer2_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, layer1_depth, layer2_depth], 
                            stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[layer2_depth]))
    
    layer3_kernel = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, layer2_depth, layer3_depth], 
                            stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[layer3_depth]))
    
    layer4_weights = tf.Variable(
        tf.truncated_normal([2 * 2 * layer3_depth, num_hidden], 
                            stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    # fully connected
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data, train=False):
        # C1
        conv = tf.nn.conv2d(data, layer1_kernel, [1, 1, 1, 1], padding='VALID')
        # S2
        pool = tf.nn.avg_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer1_biases)
        # C3
        conv = tf.nn.conv2d(hidden, layer2_kernel, [1, 1, 1, 1], padding='VALID')
        # S4
        pool = tf.nn.avg_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer2_biases)
        # C5
        conv = tf.nn.conv2d(hidden, layer3_kernel, [1, 3, 3, 1], padding='VALID')
        hidden = tf.nn.relu(conv)
        # F6
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        fully_connected = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
        # OUTPUT
        logits = tf.matmul(fully_connected, layer5_weights) + layer5_biases
        return logits
  
    # Training computation.
    logits = model(tf_train_dataset, train=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [68]:
run_graph(lenet_graph, batched_valid_and_test=False)

Initialized
Minibatch loss at step 0: 3.023194
Minibatch accuracy: 9.0%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.626792
Minibatch accuracy: 80.0%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.515088
Minibatch accuracy: 85.0%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.441104
Minibatch accuracy: 85.0%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.321787
Minibatch accuracy: 93.0%
Validation accuracy: 88.2%
Minibatch loss at step 5000: 0.490758
Minibatch accuracy: 86.0%
Validation accuracy: 88.7%
Minibatch loss at step 6000: 0.289022
Minibatch accuracy: 90.0%
Validation accuracy: 88.8%
Minibatch loss at step 7000: 0.344662
Minibatch accuracy: 87.0%
Validation accuracy: 89.2%
Minibatch loss at step 8000: 0.251160
Minibatch accuracy: 93.0%
Validation accuracy: 89.1%
Test accuracy: 95.2%


- tried something similar to LeNet5 with 7 layers (~93%)..
    - convolution 3x3, stride 1, depth of 6, padding valid
    - pool with stride of 2, padding same
    - sigmoid activation
    - convolution 3x3, stride 1, depth 16, padding valid
    - pool with stride of 2, padding same
    - sigmoid activation
    - convolution 3x3, stride 1, depth 120, padding valid
    - dense layer with 84 units
    - logits layer
- tried with relu activation, gave ~2% boost